In [3]:
import requests

# Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [5]:
def get_name_max_rate():
    """
    Функция определяет валюту с максимальным курсом 
    на основе данных https://www.cbr-xml-daily.ru/daily_json.js
    и возвращает название этой валюты.
    
    Пример JSON'а:
    {
    "Date": "2020-12-12T11:30:00+03:00",
    "PreviousDate": "2020-12-11T11:30:00+03:00",
    "PreviousURL": "\/\/www.cbr-xml-daily.ru\/archive\/2020\/12\/11\/daily_json.js",
    "Timestamp": "2020-12-13T21:00:00+03:00",
    "Valute": {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 55.0955,
            "Previous": 55.1148
        },
        ...
      }
    }
    
    >>> get_name_max_rate()
    'СДР (специальные права заимствования)'
    
    """
    
    data = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()['Valute']
    
    max_rate = 0
    max_rate_name = None
    for key, value in data.items():
        if value['Value'] > max_rate:
            max_rate = value['Value']
            max_rate_name = value['Name']
    
    # Улучшенная реализация алгоритма поиска максимума (от преподавателя)
    # max_rate_name = max(data.items(), key=lambda x: x[1]['Value'])[1]['Name']
    
    return max_rate_name

In [6]:
get_name_max_rate()

'СДР (специальные права заимствования)'

# Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [20]:
class Rate:
    def __init__(self, format_='value', diff=False):
        self.format = format_
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if self.diff:  # Когда нужно вернуть изменение курса
                    return round(response[currency]['Value'] - response[currency]['Previous'], 4)
                elif not self.diff:
                    return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')
    
    

In [21]:
r = Rate(diff=True)

In [22]:
r.eur()

-0.3586

# Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

**Комментарий по классу Designer такой:**

Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия. Комментарий про его условия Вика написала выше: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”

In [38]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)

In [50]:
class Designer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
        
        self.inernational_awards = 2
    
    def check_if_it_is_time_for_upgrade(self):
        # Добавление балла за прохождение аккредитации
        self.seniority += 1
        
        # Учёт баллов за международные премии
        sum_seniority = self.seniority + self.inernational_awards * 2
        
        # Повышение на 1 грейд за каждые 7 баллов
        if sum_seniority % 7 == 0:
            self.grade_up()
            
        return self.publish_grade()

In [51]:
sergey = Designer(name='Sergey', seniority=0)

In [52]:
for i in range(20):
    sergey.check_if_it_is_time_for_upgrade()

Sergey 1
Sergey 1
Sergey 2
Sergey 2
Sergey 2
Sergey 2
Sergey 2
Sergey 2
Sergey 2
Sergey 3
Sergey 3
Sergey 3
Sergey 3
Sergey 3
Sergey 3
Sergey 3
Sergey 4
Sergey 4
Sergey 4
Sergey 4


In [53]:
sergey = Designer(name='Sergey', seniority=0)

In [54]:
sergey.inernational_awards = 1

In [55]:
for i in range(20):
    sergey.check_if_it_is_time_for_upgrade()

Sergey 1
Sergey 1
Sergey 1
Sergey 1
Sergey 2
Sergey 2
Sergey 2
Sergey 2
Sergey 2
Sergey 2
Sergey 2
Sergey 3
Sergey 3
Sergey 3
Sergey 3
Sergey 3
Sergey 3
Sergey 3
Sergey 4
Sergey 4
